In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
#load the data, read RGB values and convert to an array of size 150, 150, 3
#convert labels to a vector of length 26
data_iter = ImageDataGenerator()
train_data = data_iter.flow_from_directory(directory='/Users/rahuldesai/LocalDocs/Projects/ASL-Translation/ASL-Datasets/Final_Dataset', target_size=(150, 150))



Found 39922 images belonging to 26 classes.


In [13]:
X = tf.placeholder(tf.float32, shape=[None, 150, 150, 3]) 
y = tf.placeholder(tf.float32, shape=[None, 26]) #Number of letters in the alphabet

In [14]:
'''
CNN Structure
1. Convolution with 32 filters (7x7 filter)
1. Relu Activation
2. Max pooling Layer - stride of 2 (2x2)
3. Convolution with 16 filters (7x7 filter)
4. Max pooling Layer - stride of 2 (2x2)
5. Conv layer with 16 filters (7x7)
6. max pool layer stride of 2 (3x3)
--flatten
5. Fully Connected Layer
6. Relu activation 
7. Output Layer with 26 outputs 
'''

Wconv1 = tf.Variable(tf.truncated_normal(shape=[7, 7, 3, 32]))
bconv1 = tf.Variable(tf.constant(shape=[32], value=0.1))
Wconv2 = tf.Variable(tf.truncated_normal(shape=[7, 7, 32, 16]))
bconv2 = tf.Variable(tf.constant(shape=[16], value=0.1))
Wconv3 = tf.Variable(tf.truncated_normal(shape=[7, 7, 16, 16]))
bconv3 = tf.Variable(tf.constant(shape=[16], value=0.1))
W1 = tf.Variable(tf.truncated_normal(shape=[2704, 1024]))
b1 = tf.Variable(tf.constant(shape=[1024], value=0.1))
W_out = tf.Variable(tf.truncated_normal(shape=[1024, 26]))
b_out = tf.Variable(tf.constant(shape=[26], value=0.1))


In [15]:
conv_layer_1 = tf.nn.conv2d(input=X, filter=Wconv1, strides=[1, 1, 1, 1], padding='SAME') + bconv1
#dimensions --> 144 x 144 x 32
relu1 = tf.nn.relu(conv_layer_1)
pool_layer_1 = tf.nn.max_pool(value=relu1 , ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#dimensions --> 72 x 72 x 32
conv_layer_2 = tf.nn.conv2d(input=pool_layer_1, filter=Wconv2, strides=[1, 1, 1, 1], padding='SAME') + bconv2
#dimensions --> 66 x 66 x 16
relu2 = tf.nn.relu(conv_layer_2)
pool_layer_2 = tf.nn.max_pool(value=relu2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#dimensions --> 33 x 33 x 16
conv_layer_3 = tf.nn.conv2d(input=pool_layer_2, filter=Wconv3, strides=[1, 1, 1, 1], padding='SAME') + bconv3
#dimensions --> 27 x 27 x 16
relu3 = tf.nn.relu(conv_layer_3)
pool_layer_3 = tf.nn.max_pool(value=relu3 , ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
#dimensions --> 13 x 13 x 16
flatten = tf.reshape(tensor=pool_layer_3, shape=[-1, 2704])
dense1 = tf.matmul(flatten, W1) + b1
relu4 = tf.nn.relu(dense1)
logits = tf.matmul(relu4, W_out) + b_out

In [16]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

#compute accuracy 
y_pred = tf.argmax(tf.nn.softmax(logits), 1) 
equalities = tf.math.equal(tf.argmax(y, 1), y_pred)
accuracy = tf.reduce_mean(tf.cast(equalities, tf.float32))

In [17]:
batch_size = 32
epochs = 3
sess = tf.Session()
sess.run(tf.global_variables_initializer())
n_batches = 10

for epoch in range(epochs):
    total_loss = 0
    total_accuracy = 0
    for batch in range(n_batches):
        x_batch, y_batch = next(train_data)
        feed_dict = {X: x_batch, y: y_batch}
        _, ls, accuracy = sess.run([optimizer, loss, accuracy], feed_dict)
        total_accuracy += accuracy 
        total_loss += ls
    print("Epoch {0}: Loss: {1} Accuracy: {2}".format(epoch, ls, total_accuracy/n_batches))


InvalidArgumentError: Input to reshape is a tensor with 184832 values, but the requested shape requires a multiple of 2704
	 [[Node: Reshape_1 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](MaxPool_5, Reshape_1/shape)]]

Caused by op 'Reshape_1', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/anaconda3/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
